In [ ]:
!pip install -U ckip-transformers #for tokenization in Traditional Chinese
!pip install TCSP #for stopwords list in Traditional Chinese

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import pandas as pd
import re
from ckip_transformers.nlp import CkipWordSegmenter
from TCSP import read_stopwords_list

In [ ]:
# read data
df = pd.read_csv('https://raw.githubusercontent.com/Letieateat/Sentiment-Analysis-on-Traditional-Chinese-Lyrics/main/dataset/TW_lyrics.csv')
df = df[df['Lyrics'] != 'Lyrics not found']

# function to clean text
def clean_lyrics(lyric):
   if pd.isna(lyric):
        return ""
   lyric = re.sub(r'[^\u4e00-\u9fff]', '', lyric)
   lyric = re.sub(r'\*+ This Lyrics is NOT for Commercial use \*+', '', lyric)
   lyric = lyric.replace("\n", "")
   return lyric

# clean the text
df['untokenized_lyrics'] = df['Lyrics'].apply(clean_lyrics)

# delete rows without lyrics after cleaning
df = df[df['untokenized_lyrics'] != '']

# initialize drivers
ws_driver  = CkipWordSegmenter(model="bert-base")

# run pipeline for tokenization
df['tokenized_lyrics']  = ws_driver(df['untokenized_lyrics'])

# remove stopwords
stopwords = read_stopwords_list()
df['final_lyrics'] = df['tokenized_lyrics'].apply(lambda x: ' '.join([word for word in x if word not in stopwords]))

# save the cleaned data to a new CSV file
df.to_csv('final_cleaned.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Inference: 100%|██████████| 5/5 [22:00<00:00, 264.03s/it]


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/Letieateat/Sentiment-Analysis-on-Traditional-Chinese-Lyrics/main/dataset/final_cleaned.csv')

In [3]:
# Calculate the total numbers of tokens
total_tokens = df['final_lyrics'].apply(len).sum()

print(f'Total number of tokens in the final_lyrics column: {total_tokens}')

Total number of tokens in the final_lyrics column: 115394
